<a href="https://colab.research.google.com/github/Apichaya-bu/229352-stat-learning-for-data2/blob/main/Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [ ]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [ ]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [ ]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [ ]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'yes':1, 'no':0}) # Write your code here


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
x = df.drop('y_new', axis=1)

# Split the data BEFORE any transformations


# Print data shape
print(y)
print(x)


0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y_new, Length: 41188, dtype: int64
       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome    y
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent   no
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent   no
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent   no
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent   no
4       56     services  married          h

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
11388,38,blue-collar,married,NaN,no,yes,no,telephone,jun,fri,42,5,999,0,nonexistent,no
10575,35,technician,single,university.degree,no,no,no,telephone,jun,tue,48,2,999,0,nonexistent,no
1305,39,management,married,university.degree,no,no,no,telephone,may,thu,1007,1,999,0,nonexistent,no
11670,37,admin.,divorced,high.school,no,yes,no,telephone,jun,fri,7,12,999,0,nonexistent,no
34916,45,admin.,married,university.degree,no,yes,yes,cellular,may,fri,300,1,999,0,nonexistent,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12148,33,blue-collar,married,basic.4y,no,yes,no,telephone,jul,tue,102,1,999,0,nonexistent,no
13031,26,admin.,single,basic.9y,no,yes,yes,cellular,jul,tue,687,2,999,0,nonexistent,yes
8228,51,self-employed,married,university.degree,NaN,yes,no,telephone,jun,tue,34,1,999,0,nonexistent,no
7075,45,technician,divorced,professional.course,no,no,no,telephone,may,thu,87,7,999,0,nonexistent,no


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [ ]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training data

x_train[num_cols_demo] = scaler.fit_transform(x_train[num_cols_demo])
x_test[num_cols_demo] = scaler.transform(x_test[num_cols_demo])


In [ ]:
x_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,-2.104691e-16,259.177829,6.654176e-17,962.214249,0.174326
std,1.000017e+00,262.765618,1.000017e+00,187.559045,0.499734
min,-2.207536e+00,0.000000,-5.683009e-01,0.000000,0.000000
25%,-7.687151e-01,103.000000,-5.683009e-01,999.000000,0.000000
50%,-1.931869e-01,179.000000,-2.015399e-01,999.000000,0.000000
75%,6.701055e-01,318.000000,1.652212e-01,999.000000,0.000000
max,5.562096e+00,4918.000000,1.483566e+01,999.000000,7.000000


Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [ ]:
print("\nOriginal X_train 'age' and 'campaign' head:")
print(x_train[num_cols_demo].head())
print("\nScaled X_train 'age' and 'campaign' head:")
print(x_train[num_cols_demo].head()) # x_train[num_cols_demo] already contains the scaled data

print("\nMean of scaled 'age' (train):", x_train[num_cols_demo]['age'].mean())
print("Std Dev of scaled 'campaign' (train):", x_train[num_cols_demo]['campaign'].std())


Original X_train 'age' and 'campaign' head:
            age  campaign
29369  0.190499  0.165221
8688  -1.248322 -0.568301
23642 -0.768715  1.999027
30110  4.315118  0.165221
20593  0.094577 -0.201540

Scaled X_train 'age' and 'campaign' head:
            age  campaign
29369  0.190499  0.165221
8688  -1.248322 -0.568301
23642 -0.768715  1.999027
30110  4.315118  0.165221
20593  0.094577 -0.201540

Mean of scaled 'age' (train): -2.104691118376489e-16
Std Dev of scaled 'campaign' (train): 1.000017342894998


**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [ ]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

## Write your code here

x_train[ord_col_demo] = imputer_ord.fit_transform(x_train[ord_col_demo])# Write your code here
x_test[ord_col_demo] = imputer_ord.transform(x_test[ord_col_demo])# Write your code here

x_train['education']

,education
29369,high.school
8688,high.school
23642,university.degree
30110,basic.4y
20593,professional.course
...,...
882,university.degree
8350,high.school
799,basic.6y
4292,high.school


- **Ordinal Encoding**

In [ ]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [ ]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

## Write your code here

x_train[ord_col_demo] = ordinal_encoder.fit_transform(x_train[ord_col_demo])  # Write your code here
x_test[ord_col_demo] = ordinal_encoder.transform(x_test[ord_col_demo])# Write your code here

Let's take a look at the imputed and ordinal-encoded `education`.

In [ ]:
print("\nOriginal X_train 'education' head:")
# Note: x_train has already been modified in place with imputed and ordinal encoded values.
# This will display the final ordinal encoded state.
print(x_train[ord_col_demo].iloc[20:25])
print("\nImputed X_train 'education' head (after imputer.transform):")
# Note: X_train_imputed_ord_demo was not created. Displaying the current (ordinal encoded) state of x_train.
print(pd.DataFrame(x_train[ord_col_demo], columns=ord_col_demo, index=x_train.index).iloc[20:25])
print("\nOrdinal Encoded X_train 'education' head:")
# Note: X_train_ord_encoded_demo was not created. Displaying the current (ordinal encoded) state of x_train.
print(pd.DataFrame(x_train[ord_col_demo], columns=ord_col_demo, index=x_train.index).iloc[20:25])


Original X_train 'education' head:
       education
33848        3.0
40710        5.0
37046        6.0
31927        5.0
31897        1.0

Imputed X_train 'education' head (after imputer.transform):
       education
33848        3.0
40710        5.0
37046        6.0
31927        5.0
31897        1.0

Ordinal Encoded X_train 'education' head:
       education
33848        3.0
40710        5.0
37046        6.0
31927        5.0
31897        1.0


**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [ ]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(x_train[nom_col_demo])

x_train[nom_col_demo] = imputer_nom.fit_transform(x_train[nom_col_demo])
x_test[nom_col_demo] = imputer_nom.transform(x_test[nom_col_demo])

- **Nominal Encoding**

In [ ]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

x_train_onehot_encoded_demo = onehot_encoder.fit_transform(x_train[['job']])
x_test_onehot_encoded_demo = onehot_encoder.transform(x_test[['job']])

x_train_onehot_encoded_demo.shape

(28831, 11)

In [ ]:
x_train_onehot_encoded_demo

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
x_train = pd.concat([x_train.reset_index(drop=True),
                     pd.DataFrame(x_train_onehot_encoded_demo, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)
x_train = x_train.drop('job', axis=1)

In [ ]:
x_test = pd.concat([x_test.reset_index(drop=True),
                     pd.DataFrame(x_test_onehot_encoded_demo, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-0.193187,blue-collar,married,6.0,no,yes,no,telephone,jun,fri,42,0.898743,999,0,nonexistent,no,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.480951,technician,single,6.0,no,no,no,telephone,jun,tue,48,-0.201540,999,0,nonexistent,no,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.097265,management,married,6.0,no,no,no,telephone,may,thu,1007,-0.568301,999,0,nonexistent,no,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.289108,admin.,divorced,4.0,no,yes,no,telephone,jun,fri,7,3.466071,999,0,nonexistent,no,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.478263,admin.,married,6.0,no,yes,yes,cellular,may,fri,300,-0.568301,999,0,nonexistent,no,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,-0.672794,blue-collar,married,1.0,no,yes,no,telephone,jul,tue,102,-0.568301,999,0,nonexistent,no,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12353,-1.344243,admin.,single,3.0,no,yes,yes,cellular,jul,tue,687,-0.201540,999,0,nonexistent,yes,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12354,1.053791,self-employed,married,6.0,NaN,yes,no,telephone,jun,tue,34,-0.568301,999,0,nonexistent,no,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
12355,0.478263,technician,divorced,5.0,no,no,no,telephone,may,thu,87,1.632266,999,0,nonexistent,no,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
print("\nOriginal X_train 'job' head:")
# The 'job' column was dropped from x_train in a previous step, so it cannot be directly accessed here.
# The original state would require a copy of x_train before any modifications.
# print(x_train[nom_col_demo].iloc[40:45]) # This would cause a KeyError

print("\nImputed X_train 'job' head (after imputer.transform):")
# The imputed 'job' column was part of x_train before being dropped and one-hot encoded.
# The variable X_train_imputed_nom_demo was never explicitly created.
# print(pd.DataFrame(X_train_imputed_nom_demo, columns=nom_col_demo, index=x_train.index).iloc[40:45]) # This would cause a NameError

print("\nOne-Hot Encoded X_train 'job' shape:", x_train_onehot_encoded_demo.shape)
print("First 5 rows of One-Hot Encoded X_train 'job':")
# Using the correctly defined x_train_onehot_encoded_demo
print(pd.DataFrame(x_train_onehot_encoded_demo, columns=onehot_encoder.get_feature_names_out(nom_col_demo), index=x_train.index).iloc[40:45])


Original X_train 'job' head:

Imputed X_train 'job' head (after imputer.transform):

One-Hot Encoded X_train 'job' shape: (28831, 11)
First 5 rows of One-Hot Encoded X_train 'job':
    job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  job_management  job_retired  job_self-employed  job_services  job_student  job_technician  job_unemployed
40         0.0              0.0               0.0            1.0             0.0          0.0                0.0           0.0          0.0             0.0             0.0
41         0.0              1.0               0.0            0.0             0.0          0.0                0.0           0.0          0.0             0.0             0.0
42         0.0              1.0               0.0            0.0             0.0          0.0                0.0           0.0          0.0             0.0             0.0
43         0.0              0.0               0.0            0.0             0.0          0.0                0.0           0.0    

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [ ]:
# --- YOUR CODE FOR EXERCISE 1 STARTS HERE ---
import pandas as pd
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'
df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)

df['y_new'] = df['y'].map({'yes':1, 'no':0})

y = df['y_new']
x = df.drop(columns=['y_new', 'y', 'duration'], errors='ignore')


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Define feature groups

numerical_features = ['age', 'campaign', 'pdays', 'previous']

ordinal_features = ['education']
education_categories = [
    ['illiterate', 'basic.4y', 'basic.6y', 'basic.9y',
     'high.school', 'professional.course', 'university.degree', 'masters', 'doctorate']
]

nominal_features = ['job', 'marital', 'default', 'housing', 'loan',
                    'contact', 'month', 'day_of_week', 'poutcome']

# Copy dataset

X_train_copy = x_train.copy()
X_test_copy = x_test.copy()

# Numerical → Standard Scaling

scaler = StandardScaler()
X_train_copy[numerical_features] = scaler.fit_transform(X_train_copy[numerical_features])
X_test_copy[numerical_features] = scaler.transform(X_test_copy[numerical_features])

# Ordinal → Ordinal Encoding

imputer_ord = SimpleImputer(strategy='most_frequent')
X_train_copy[ordinal_features] = imputer_ord.fit_transform(X_train_copy[ordinal_features])
X_test_copy[ordinal_features] = imputer_ord.transform(X_test_copy[ordinal_features])

ordinal_encoder = OrdinalEncoder(categories=education_categories)
X_train_copy[ordinal_features] = ordinal_encoder.fit_transform(X_train_copy[ordinal_features])
X_test_copy[ordinal_features] = ordinal_encoder.transform(X_test_copy[ordinal_features])

# Nominal → One-Hot Encoding (with Imputation for all)

imputer_nom = SimpleImputer(strategy='most_frequent')
X_train_copy[nominal_features] = imputer_nom.fit_transform(X_train_copy[nominal_features])
X_test_copy[nominal_features] = imputer_nom.transform(X_test_copy[nominal_features])

ohe = OneHotEncoder(drop="first", sparse_output=False, handle_unknown='ignore')

ohe_train = ohe.fit_transform(X_train_copy[nominal_features])
ohe_train_df = pd.DataFrame(ohe_train,
                            columns=ohe.get_feature_names_out(nominal_features),
                            index=X_train_copy.index)

ohe_test = ohe.transform(X_test_copy[nominal_features])
ohe_test_df = pd.DataFrame(ohe_test,
                           columns=ohe.get_feature_names_out(nominal_features),
                           index=X_test_copy.index)

X_train_copy = pd.concat([X_train_copy.drop(columns=nominal_features), ohe_train_df], axis=1)
X_test_copy = pd.concat([X_test_copy.drop(columns=nominal_features), ohe_test_df], axis=1)

# Train Logistic Regression

model = LogisticRegression(max_iter=2000)
model.fit(X_train_copy, y_train)

# Predict + Classification Report

y_pred = model.predict(X_test_copy)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.91      0.99      0.94     10968
           1       0.66      0.19      0.29      1389

    accuracy                           0.90     12357
   macro avg       0.78      0.59      0.62     12357
weighted avg       0.88      0.90      0.87     12357



## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Fashion-MNIST Train data shape: (60000, 28, 28)
Fashion-MNIST Train labels shape: (60000,)
Fashion-MNIST Test data shape: (10000, 28, 28)
Fashion-MNIST Test labels shape: (10000,)


In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

First image [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0
    0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62
   54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134
  144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178
  107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216
  216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   0   0   0   0   0   1   1   1   

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


